# Get NPPES 2020 Data

In [1]:
# import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

### Identify the columns we need

In [20]:
# import the file header for the NPI data as a dataframe
nppes_header = pd.read_csv('../data/nppes/npidata_pfile_20050523-20210207_FileHeader.csv')

In [21]:
# write the file header columns to a list
nppes_headers_list = nppes_header.columns.to_list()

In [22]:
# write that list back to a dataframe in a column called 'variables'
nppes_headers_list = pd.DataFrame(nppes_headers_list, columns = ['variables'])

In [23]:
# Check out the variables generated
nppes_headers_list

,variables
0,NPI
1,Entity Type Code
2,Replacement NPI
3,Employer Identification Number (EIN)
4,Provider Organization Name (Legal Business Name)
...,...
325,Healthcare Provider Taxonomy Group_12
326,Healthcare Provider Taxonomy Group_13
327,Healthcare Provider Taxonomy Group_14
328,Healthcare Provider Taxonomy Group_15


In [6]:
# Select only the columns we want
nppes_headers_vars = nppes_headers_list[nppes_headers_list['variables'].isin(['Provider Organization Name (Legal Business Name)',
'Provider Last Name (Legal Name)',
'Provider First Name',
'Provider Middle Name',
'Provider Name Prefix Text',
'Provider Name Suffix Text',
'Provider Credential Text',
'Provider First Line Business Mailing Address',
'Provider Second Line Business Mailing Address',
'Provider Business Mailing Address City Name',
'Provider Business Mailing Address State Name',
'Provider Business Mailing Address Postal Code']) == True]
# Maggi will get the ‘Healthcare Provider Taxonomy Code*’ fields, with primary type, per NPI and we'll join that later

In [7]:
# Check out the variables
nppes_headers_vars

,variables
4,Provider Organization Name (Legal Business Name)
5,Provider Last Name (Legal Name)
6,Provider First Name
7,Provider Middle Name
8,Provider Name Prefix Text
9,Provider Name Suffix Text
10,Provider Credential Text
20,Provider First Line Business Mailing Address
21,Provider Second Line Business Mailing Address
22,Provider Business Mailing Address City Name


In [8]:
# Write it to a list
nppes_headers_vars = nppes_headers_vars.variables.to_list()

In [9]:
# Check out that list
nppes_headers_vars

['Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider First Line Business Mailing Address',
 'Provider Second Line Business Mailing Address',
 'Provider Business Mailing Address City Name',
 'Provider Business Mailing Address State Name',
 'Provider Business Mailing Address Postal Code']

### Create a database to hold all the things

In [5]:
# create a database or connect to an existing one
db = sqlite3.connect('../data/hop_teaming.sqlite')

In [17]:
# if you need to rewrite the database...
# cursor = db.cursor()

In [18]:
# Drop the table and return a line that says that it's gone
# cursor.execute("DROP TABLE nppes")
# print("Table dropped...")

Table dropped...


In [26]:
# Iterate over the dataframe, using only the selected columns, and build a table
# With a chunksize of 5000, it will take 1343 loops to build the database
# There will be a pink warning re: column types which can probably be disregarded
for chunk in tqdm(pd.read_csv('../data/nppes/npidata_pfile_20050523-20210207.csv', usecols = nppes_headers_vars, chunksize = 5000)):
    #chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('nppes', db, if_exists = 'append', index = False)            # Append the chunk to a calls table

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# Write a test query
query = '''
SELECT *
FROM nppes
LIMIT 10
'''

In [7]:
# Write the results to a test dataframe
test = pd.read_sql(query, db)

In [8]:
# Check out the test dataframe
test

,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code
0,None,WIEBE,DAVID,A,None,None,M.D.,PO BOX 2168,None,KEARNEY,NE,688482168.0
1,None,PILCHER,WILLIAM,C,DR.,None,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0
2,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,None,3418 VILLAGE DR,None,FAYETTEVILLE,NC,283044552.0
3,None,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,None,ATHENS,TX,757513446.0
4,None,GRESSOT,LAURENT,None,DR.,None,M.D.,17323 RED OAK DR,None,HOUSTON,TX,770901243.0
5,COLLABRIA CARE,None,None,None,None,None,None,414 S JEFFERSON ST,None,NAPA,CA,945594515.0
6,None,ADUSUMILLI,RAVI,K,None,None,MD,2940 N MCCORD RD,None,TOLEDO,OH,436151753.0
7,None,WORTSMAN,SUSAN,None,None,None,MA-CCC,68 ROCKLEDGE RD,APT 1C,HARTSDALE,NY,105303455.0
8,None,BISBEE,ROBERT,None,DR.,None,MD,5219 CITY BANK PKWY STE 214,None,LUBBOCK,TX,794073537.0
9,None,SUNG,BIN,SHENG,None,None,M. D.,600 JEFFERSON ST STE 301,None,LAFAYETTE,LA,705016987.0


In [9]:
db.close()